In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib as plt
import random
from sklearn.utils import shuffle
import pickle

In [2]:
#ds, ds_info = tfds.load('omniglot', split=['train','test'],with_info=True, as_supervised = False)
#ds_info

In [3]:
test = pickle.load(open("TestDF_ds.pickle", "rb"))
train = pickle.load(open("TrainDF_ds.pickle", "rb"))

In [4]:
#len(ds[0])

In [5]:
#len(ds[1])

In [6]:
trainDF = pd.DataFrame(train)
testDF = pd.DataFrame(test)

In [7]:
trainDF = trainDF[:8000]
testDF = testDF[:2000]

In [8]:
def make_positive_pairs(df):
    
    list_image = []    
    pairList= []
    
    uniqueAlph = df['alphabet'].unique()
    
    for i in range(len(uniqueAlph)):
        alphDF = df[df['alphabet'] == i]
        charUnique = alphDF['alphabet_char_id'].unique()
        for i in range(len(charUnique)):
            dfImage = alphDF[alphDF['alphabet_char_id'] == charUnique[i]]
            
            imagedf = dfImage['image'].copy()
            imageList = imagedf.values.tolist()
            
            
            list_image.append(imageList[0])
            
        x = 0 
        pair_list = []
        pair_target = []
        list_image2 = list_image.copy()
        for i in range(len(list_image)):
            for x in range(len(list_image)):
                pair_list.append([list_image[i], list_image2[x]])
                pair_target.append(1.0)
                
                
        random.shuffle(pair_list)       
        
            
            
            
    return pair_list, pair_target
        

In [9]:
def get_pairs(data):
        
    
    df = data.sort_values(by=['alphabet'])
    
    df_pairs_1 = df[df.index % 2 != 0]
    
    df_pairs_2 = df[df.index % 2 != 1]
    
    df_pairs_1 = shuffle(df_pairs_1)
    
    imagedf = df_pairs_1['image'].copy()
    imageList = imagedf.values.tolist()
    
    alphabetdf = df_pairs_1['alphabet'].copy()
    alphabetList = alphabetdf.values.tolist()
    
    imagedf2 = df_pairs_2['image'].copy()
    imageList2 = imagedf.values.tolist()
    
    alphabetdf2 = df_pairs_2['alphabet'].copy()
    alphabetList2 = alphabetdf.values.tolist()
    
    imageList2 = shuffle(imageList2)
    imageList = shuffle(imageList)
    alphabetList = shuffle(alphabetList)
    alphabetList2 = shuffle(alphabetList2)
    
    trainList = []
    testList = []
    
    for i in range(len(alphabetList)):
        loopList = []
        pos = 1.0
        neg = 0.0
        if alphabetList[i] == alphabetList2[i]:
            loopList.append(imageList[i].astype(float))
            loopList.append(imageList2[i].astype(float))
            testList.append(pos)
            trainList.append(loopList)
        else:
            loopList.append(imageList[i].astype(float))
            loopList.append(imageList2[i].astype(float))
            testList.append(neg)
            trainList.append(loopList)
    
    return trainList, testList

In [10]:
def count_neg_pairs(list_bin):
    x = 1
    count = []
    for i in range(len(list_bin)):
        if list_bin[i] == 0.0:
            count.append(x)
        else:
            pass
    print(len(count))

In [11]:
def combine_pairs(train1, test1, train2, test2):
    for i in range(len(train2)):
        train1.append(train2[i])
    
    
    for i in range(len(test2)):
        test1.append(test2[i])
    
    shuffleList = list(zip(train1, test1))
                       
    random.shuffle(shuffleList)
    
    train1, test1 = zip(*shuffleList)
                       
    train1 = np.array(train1)
    test1 = np.array(test1)
    
    train1/225

    
    return train1, test1

In [12]:
trainX, trainY = get_pairs(trainDF)

In [13]:
count_neg_pairs(trainY)

3874


In [14]:
trainXPos, trainYPos = make_positive_pairs(trainDF)

In [15]:
trainXPos = trainXPos[0:20000]

In [16]:
trainYPos = trainYPos[0:20000]

In [17]:
trainXFinal, trainYFinal= combine_pairs(trainXPos, trainYPos, trainX, trainY)

In [18]:
len(trainYFinal)

24000

In [19]:
len(trainXFinal)

24000

In [20]:
#trainYFinal = trainYFinal.reshape(69640, 1)

In [21]:
count_neg_pairs(trainYFinal)

3874


test data preprocessing

In [22]:
testX, testY = get_pairs(testDF)

In [23]:
count_neg_pairs(testY)

952


In [24]:
testXPos, testYPos = make_positive_pairs(testDF)

In [25]:
count_neg_pairs(testYPos)

0


In [26]:
testXFinal, testYFinal= combine_pairs(testXPos, testYPos, testX, testY)

In [27]:
testXPos = testXPos[:5000]
testYPos = testYPos[:5000]

In [28]:
'''
import pickle
pickle_out = open("trainXFinal.pickle", "wb")
pickle.dump(trainXFinal, pickle_out)
pickle_out.close()

pickle_out = open("trainYPos.pickle", "wb")
pickle.dump(trainYPos, pickle_out)
pickle_out.close()

pickle_out = open("testXFinal.pickle", "wb")
pickle.dump(testXFinal, pickle_out)
pickle_out.close()

pickle_out = open("testYFinal.pickle", "wb")
pickle.dump(testYFinal, pickle_out)
pickle_out.close()


'''

'\nimport pickle\npickle_out = open("trainXFinal.pickle", "wb")\npickle.dump(trainXFinal, pickle_out)\npickle_out.close()\n\npickle_out = open("trainYPos.pickle", "wb")\npickle.dump(trainYPos, pickle_out)\npickle_out.close()\n\npickle_out = open("testXFinal.pickle", "wb")\npickle.dump(testXFinal, pickle_out)\npickle_out.close()\n\npickle_out = open("testYFinal.pickle", "wb")\npickle.dump(testYFinal, pickle_out)\npickle_out.close()\n\n\n'

In [29]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
#import wandb
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, Lambda, Dot
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

In [30]:
#X_train, X_test, y_train, y_test = train_test_split(trainXFinal, trainYFinal, test_size=0.20, random_state=42)


In [31]:
lenX = round(len(trainXFinal)*0.20)
lenY = round(len(trainYFinal)*0.20)
X_train = trainXFinal[lenX:]
X_test = trainYFinal[lenX:]
y_train = trainXFinal[:lenY]
y_test = trainYFinal[:lenY]

In [32]:
epochs = 20
batch_size = 16
margin = 1  # Margin for constrastive loss.

In [33]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))


input = layers.Input((105, 105, 3))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(16, (3, 3), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(64, activation="tanh")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((105, 105, 3))
input_2 = layers.Input((105, 105, 3))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [34]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

  Arguments:
      margin: Integer, defines the baseline for distance for which pairs
              should be classified as dissimilar. - (default is 1).

  Returns:
      'constrastive_loss' function with data ('margin') attached.
  """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

      Arguments:
          y_true: List of labels, each label is of type float32.
          y_pred: List of predictions of same length as of y_true,
                  each label is of type float32.

      Returns:
          A tensor containing constrastive loss as floating point value.
      """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [35]:
siamese.compile(loss=loss(margin=margin), optimizer="Adam", metrics=["accuracy"])
siamese.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 105, 105, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 105, 105, 3) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 64)           1258540     input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           model[0][0]                

In [36]:
history = siamese.fit(
    [X_train[:,0], X_train[:,1]],
    X_test[:],
    validation_data=([y_train[:,0], y_train[:,1]], y_test[:]),
    batch_size=batch_size,
    epochs=epochs,
)

Epoch 1/20
1200/1200 [==============================] - 132s 108ms/step - loss: 0.1763 - accuracy: 0.7559 - val_loss: 0.2259 - val_accuracy: 0.6310
Epoch 2/20
1200/1200 [==============================] - 128s 107ms/step - loss: 0.1087 - accuracy: 0.8503 - val_loss: 0.1349 - val_accuracy: 0.8400
Epoch 3/20
1200/1200 [==============================] - 122s 101ms/step - loss: 0.0861 - accuracy: 0.8808 - val_loss: 0.1551 - val_accuracy: 0.8396
Epoch 4/20
1200/1200 [==============================] - 121s 101ms/step - loss: 0.0749 - accuracy: 0.8979 - val_loss: 0.0780 - val_accuracy: 0.8994
Epoch 5/20
1200/1200 [==============================] - 123s 102ms/step - loss: 0.0745 - accuracy: 0.8961 - val_loss: 0.0552 - val_accuracy: 0.9329
Epoch 6/20
1200/1200 [==============================] - 122s 102ms/step - loss: 0.0647 - accuracy: 0.9140 - val_loss: 0.1474 - val_accuracy: 0.8417
Epoch 7/20
1200/1200 [==============================] - 122s 101ms/step - loss: 0.0766 - accuracy: 0.8965 - val_

In [37]:
y_train[0][0] = y_train[0][0].reshape(-1, 105, 105, 3)

In [38]:
y_train[0][1] = y_train[0][1].reshape(-1, 105, 105, 3)

In [ ]:
prediction = siamese.predict([y_train[:, 0].reshape(-1, 105, 105, 3), y_train[:, 1].reshape(-1, 105, 105, 3)])

In [47]:
def get_result(X_value, y_value):
    predictions = []
    count = 0
    for i in range(len(X_value)):
        prediction = siamese.predict([X_value[i][0].reshape(-1, 105, 105, 3), X_value[i][1].reshape(-1, 105, 105, 3)])
        if prediction > 0.5:
            predictions.append(1)
        else:
            predictions.append(0)
    for i in range(len(testXFinal)):
        if predictions[i] == y_value[i]:
            counts = counts+1
    accurracy = (counts/len(testXFinal))*100

In [ ]:
test_acc = get_result(testXFinal, testYFinal)

In [ ]:
train_acc = get_result(trainXFinal, trainYFinal)

In [53]:
testXFinal = np.array(testXFinal)
trainYFinal = np.array(trainYFinal)
testYFinal = np.array(testYFinal)



In [ ]:
testXFinal = np.array(testXFinal)

In [ ]:
mixX = np.concatenate(trainXFinal[:100],trainXFinal[:100])
mixY = np.concatenate(testYFinal[:100],trainYFinal[:100])

In [ ]:
mix_acc = get_result(mixX, mixY)

In [ ]:
Print("train accuracy is {}%".format(train_acc))

In [ ]:
Print("test accuracy is {}%".format(test_acc))

In [ ]:
Print("mix of train and test accuracy is {}%".format(mix_acc))